In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

# Подготовка валидационной выборки (удаление ev_type=2 (skip))

Запустила обучение на новом бинарнике, результаты предыдущего запуска перенесла в ```~/TRG-78708/new_hpr_online_stand_info/installs/stand```

## Валидационные файлы

In [5]:
valid_path = '/home/eka.volkova/TRG-78708/orig_data_bin/installs/csv/'

In [6]:
files = [
    "install_20220613000000.csv",
    "install_20220613042000.csv",
    "install_20220613080500.csv",
    "install_20220613120000.csv",
    "install_20220613155500.csv",
    "install_20220613200000.csv",
    "install_20220614000500.csv",
    "install_20220614040000.csv",
    "install_20220614080000.csv",
    "install_20220614115500.csv",
    "install_20220614155500.csv",
    "install_20220614200000.csv",
    "install_20220615000000.csv",
    "install_20220615035500.csv",
    "install_20220615080000.csv"]

In [7]:
n = 0
valid_file = pd.DataFrame({'cr_model': []})
for file in files:
    df = pd.read_csv(f'{valid_path}/{file}', sep=';')
    df_shape = df.shape[0]
    valid_file = pd.concat([valid_file, df], ignore_index=True)
    n += df_shape

/usr/local/trgml/dev/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (75) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/trgml/dev/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (63,75) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/trgml/dev/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
valid_file.shape

(5190825, 127)

## Удаляем строки с ev_type=2 и к полученной валидационной выборке присоединяем оценки из нового стенда

In [15]:
valid_estim_old = pd.read_csv('~/TRG-78708/hpr_online_stand_info/installs/stand/prod_installs(lr)', sep=';')

valid_estim = valid_file[['priced', 'sex']].join(valid_estim_old)

valid_estim = valid_estim[~(valid_estim['ev_type']==2)].reset_index(drop=True)

valid_estim.drop(columns=['pr', 'ev_type'], inplace=True)

valid_estim_new = pd.read_csv('~/TRG-78708/new_hpr_online_stand_info/installs/stand/prod_installs(lr)', sep=';')

valid_estim = valid_estim.join(valid_estim_new)

valid_estim.shape

In [16]:
valid_estim['ev_type'].value_counts()

0    4869160
1     321647
Name: ev_type, dtype: int64

---
# Расчет суммарного logloss в разрезе adequacy_keys ```'priced', 'sex'```

## ```adequacy/prod.csv```
- На версии нового стенда, в которой ЕСТЬ разбиения по ```range``` 

In [17]:
adequacy_prod = pd.read_csv('~/TRG-78708/new_hpr_online_stand_info/installs/stand/prod.csv', sep=';')

In [2]:
adequacy_prod = pd.read_csv('~/TRG-78708/new_hpr_online_stand_info/installs/stand/prod.csv', sep=';')

In [19]:
adequacy_prod = adequacy_prod.groupby(['priced', 'sex'])[['negative', 'positive', 'logloss']].sum()

In [20]:
adequacy_prod.rename(columns={'negative': 'Clicks_all', 'positive': 'Installs', 'logloss': 'Logloss'}, inplace=True)

In [3]:
adequacy_prod

,priced,sex,range,suite,negative,positive,e_positive,v_positive,logloss
0,oCPM_inapp,2,0.0001:0.0005,prod,1,0,0.000469,0.000469,0.000469
1,CPC,0,0.0500:0.1000,prod,1,0,0.053077,0.050260,0.054538
2,oCPM_inapp,2,0.5000:1.0000,prod,2,1,1.060344,0.496897,1.494289
3,CPC,0,0.0001:0.0005,prod,1,0,0.000119,0.000119,0.000119
4,oCPM_inapp,2,0.0005:0.0010,prod,3,0,0.002297,0.002295,0.002298
...,...,...,...,...,...,...,...,...,...
116,CPC,1,0.0001:0.0005,prod,937,0,0.285109,0.285011,0.285158
117,CPM,2,0.1000:0.5000,prod,163,25,27.578028,22.070889,70.013967
118,CPI,2,0.0010:0.0050,prod,12238,49,39.626500,39.482798,318.230211
119,CPC,1,0.0500:0.1000,prod,945,37,63.288868,58.884320,161.746823


In [110]:
adequacy_prod

Clicks_all  Installs        Logloss
priced     sex                                     
CPC        0           131         2      10.664170
           1         29147       268    1398.594988
           2         51848       515    2633.750102
CPI        0        116052      4454   15395.449546
           1        142633      7427   23859.434629
           2        112222      4893   15964.595294
CPM        0          2585         8      57.482797
           1         28915       154     874.866106
           2         24305       154     772.391451
oCPM       0        353222      5271   20322.678702
           1       2007997    149989  436324.803001
           2       2318563    148461  453714.337538
oCPM_inapp 0          1406        19      76.187666
           1           841        17      68.584444
           2           940        15      59.816078

# Расчет logloss для сравнения с logloss из логов нового hpr_online_stand'а

In [51]:
logloss_cr = adequacy.copy()

In [53]:
logloss_cr = logloss_cr.join(pd.get_dummies(logloss_cr['ev_type']))
logloss_cr.rename(columns={0: 'Clicks', 1: 'Installs'}, inplace=True)

In [56]:
logloss_cr['Logloss'] = -(logloss_cr['ev_type']*np.log(logloss_cr['pr']) + (1-logloss_cr['ev_type'])*np.log(1-logloss_cr['pr']))

In [57]:
logloss_cr.head()

,priced,sex,pr,ev_type,Clicks,Installs,Logloss
0,oCPM,1.0,0.061474,0,1,0,0.063445
1,oCPM,0.0,0.251305,0,1,0,0.289424
2,oCPM,1.0,0.003210,0,1,0,0.003215
3,oCPM,1.0,0.055638,0,1,0,0.057246
4,oCPM,1.0,0.209112,0,1,0,0.234599


In [245]:
# Если надо посчитать logloss по отдельному срезу 
# logloss_cr = logloss_cr.query('priced == "CPC" and sex == 0.0 and range_right == "(0.0050, 0.0100]"')

In [58]:
data_cr = logloss_cr.agg({'Clicks': 'sum', 'Installs': 'sum', 'pr': 'sum', 'Logloss': 'sum'})
data_cr = pd.DataFrame(data_cr).T
data_cr

,Clicks,Installs,pr,Logloss
0,4869160.0,321647.0,321588.78799,971533.641584


In [60]:
'Общее количество событий НЕ совпадает с количеством строк в валидационной выборке {:_.0f} на 18 строк (SKIP)'\
.format(data_cr[['Clicks', 'Installs']].T.sum()[0])

'Общее количество событий НЕ совпадает с количеством строк в валидационной выборке 5_190_807 на 18 строк (SKIP)'

## Расчет

In [49]:
def prepare_rate(rate, division):
    rate_copy = rate.copy() / division
    rate_copy[rate_copy >= 1] = np.nan
    rate_copy[rate_copy <= 0] = np.nan
    return rate_copy

In [50]:
def entropy(pr):
    return -(1.0 - pr) * np.log(1.0 - pr) - pr * np.log(pr)

In [61]:
data_cr['CR'] = data_cr['Installs'] * 100 / (data_cr['Clicks'] + data_cr['Installs']) 
data_cr['CR'] = prepare_rate(data_cr['CR'], 100) 
data_cr

,Clicks,Installs,pr,Logloss,CR
0,4869160.0,321647.0,321588.78799,971533.641584,0.061965


In [62]:
data_cr['Logloss_cum'] = data_cr['Logloss']/(entropy(data_cr['CR'])[0] * (data_cr['Clicks'][0] + data_cr['Installs'][0]))
data_cr

,Clicks,Installs,pr,Logloss,CR,Logloss_cum
0,4869160.0,321647.0,321588.78799,971533.641584,0.061965,0.805563


In [65]:
# Метрики из лога нового стенда
!tail -n6 ~/TRG-78708/new_hpr_online_stand_info/installs/logs/hpr_stand.log

prod	valid	auc = 0.82985
prod-new	valid	auc = 0.74095
prod	valid	logloss = 0.80556
prod-new	valid	logloss = 1.3639

14.07 10:01:06.877 [notice] *516736: main: exited


# Расчет logloss'а в целом по выборке (без деления на срезы)